In [1]:
import pandas as pd
import numpy as np

Read data file

In [2]:
transactions_df = pd.read_csv('data.csv',encoding='ISO-8859-1')

In [3]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Drop Transactions with missing customer IDs

In [4]:
transactions_df = transactions_df[transactions_df['CustomerID'].notnull()].copy()

Convert Invoice Date column from a string column to a datetime column

In [5]:
transactions_df['InvoiceDate'] = pd.to_datetime(transactions_df['InvoiceDate'])

In [6]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406829 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    406829 non-null  object        
 1   StockCode    406829 non-null  object        
 2   Description  406829 non-null  object        
 3   Quantity     406829 non-null  int64         
 4   InvoiceDate  406829 non-null  datetime64[ns]
 5   UnitPrice    406829 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      406829 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.9+ MB


Get the most recent transaction made by a customer in the dataset

In [7]:
most_recent_transaction = transactions_df['InvoiceDate'].max()

In [8]:
most_recent_transaction

Timestamp('2011-12-09 12:50:00')

Create a dataframe containing the number of days elasped since the most recent transaction for each user

In [9]:
# Compute the lastest transaction for each user
latest_transactions_per_user = transactions_df.groupby('CustomerID')['InvoiceDate'].max()

recency_df = latest_transactions_per_user.reset_index()
recency_df['recency'] = recency_df['InvoiceDate'].apply(lambda date: (most_recent_transaction - date).days)

In [10]:
recency_df.head()

,CustomerID,InvoiceDate,recency
0,12346.0,2011-01-18 10:17:00,325
1,12347.0,2011-12-07 15:52:00,1
2,12348.0,2011-09-25 13:13:00,74
3,12349.0,2011-11-21 09:51:00,18
4,12350.0,2011-02-02 16:01:00,309


Create a dataframe containing the number of transactions made for each user

In [11]:
# Calculate the number of invididual invoices generated for each customer
num_transactions_per_user = transactions_df.groupby('CustomerID')['InvoiceNo'].unique().apply(lambda lst: len(lst))

frequency_df = num_transactions_per_user.reset_index().rename(columns={'InvoiceNo':'frequency'})

In [12]:
frequency_df

,CustomerID,frequency
0,12346.0,2
1,12347.0,7
2,12348.0,4
3,12349.0,1
4,12350.0,1
...,...,...
4367,18280.0,1
4368,18281.0,1
4369,18282.0,3
4370,18283.0,16


Create a dataframe containing the total amount spent for each customer

In [13]:
# Create a new column containing the total amount spent for each item
transactions_df['Total'] = transactions_df['Quantity'] * transactions_df['UnitPrice']

# Now, aggregate total spend by customer
monetary_df = transactions_df.groupby('CustomerID')['Total'].sum().reset_index().rename(columns={'Total':'monetary'})

In [14]:
monetary_df

,CustomerID,monetary
0,12346.0,0.00
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4367,18280.0,180.60
4368,18281.0,80.82
4369,18282.0,176.60
4370,18283.0,2094.88


Merge the three dataframes into one dataframe containing each figure

In [15]:
rfm_df = pd.merge(pd.merge(recency_df, frequency_df, on='CustomerID'), monetary_df, on='CustomerID').drop('InvoiceDate',axis=1)

In [16]:
rfm_df

,CustomerID,recency,frequency,monetary
0,12346.0,325,2,0.00
1,12347.0,1,7,4310.00
2,12348.0,74,4,1797.24
3,12349.0,18,1,1757.55
4,12350.0,309,1,334.40
...,...,...,...,...
4367,18280.0,277,1,180.60
4368,18281.0,180,1,80.82
4369,18282.0,7,3,176.60
4370,18283.0,3,16,2094.88


Calcuate the recency, frequency, and monetary scores for each user by binning the scores into evenly spaced buckets. 

In [17]:
rfm_df['recency_score'] = pd.cut(rfm_df['recency'], bins=5, labels=[5,4,3,2,1]).astype(int)
rfm_df['frequency_score'] = pd.cut(rfm_df['frequency'], bins=5, labels=[1,2,3,4,5]).astype(int)
rfm_df['monetary_score'] = pd.cut(rfm_df['monetary'], bins=5, labels=[1,2,3,4,5]).astype(int)

In [18]:
rfm_df

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score
0,12346.0,325,2,0.00,1,1,1
1,12347.0,1,7,4310.00,5,1,1
2,12348.0,74,4,1797.24,5,1,1
3,12349.0,18,1,1757.55,5,1,1
4,12350.0,309,1,334.40,1,1,1
...,...,...,...,...,...,...,...
4367,18280.0,277,1,180.60,2,1,1
4368,18281.0,180,1,80.82,3,1,1
4369,18282.0,7,3,176.60,5,1,1
4370,18283.0,3,16,2094.88,5,1,1


Compute weighted RFM score for each user

In [19]:
rfm_df['rfm_score'] = (0.35*rfm_df['recency_score'] + 0.35*rfm_df['frequency_score'] + 0.40*rfm_df['monetary_score'])
rfm_df['rfm_score'] = rfm_df['rfm_score'].round(2)

In [20]:
rfm_df

,CustomerID,recency,frequency,monetary,recency_score,frequency_score,monetary_score,rfm_score
0,12346.0,325,2,0.00,1,1,1,1.10
1,12347.0,1,7,4310.00,5,1,1,2.50
2,12348.0,74,4,1797.24,5,1,1,2.50
3,12349.0,18,1,1757.55,5,1,1,2.50
4,12350.0,309,1,334.40,1,1,1,1.10
...,...,...,...,...,...,...,...,...
4367,18280.0,277,1,180.60,2,1,1,1.45
4368,18281.0,180,1,80.82,3,1,1,1.80
4369,18282.0,7,3,176.60,5,1,1,2.50
4370,18283.0,3,16,2094.88,5,1,1,2.50


Segment customers based on their RFM scores
* High value customers are customers with RFM scores greater than 3
* Medium value customers are customers with RFM scores between 2 and 3
* Low value customers are customers with RFM scores less than 2

In [21]:
def segment_customer(score):
    if score >= 3:
        return 'High'
    elif score >= 2 and score < 3:
        return 'Medium'
    else:
        return 'Low'

rfm_df['segment'] = rfm_df['rfm_score'].apply(segment_customer)

In [22]:
rfm_df[['CustomerID','rfm_score','segment']]

,CustomerID,rfm_score,segment
0,12346.0,1.10,Low
1,12347.0,2.50,Medium
2,12348.0,2.50,Medium
3,12349.0,2.50,Medium
4,12350.0,1.10,Low
...,...,...,...
4367,18280.0,1.45,Low
4368,18281.0,1.80,Low
4369,18282.0,2.50,Medium
4370,18283.0,2.50,Medium


Get a count of customers by segment

In [23]:
rfm_df.groupby('segment')['CustomerID'].count()

segment
High        14
Low       1050
Medium    3308
Name: CustomerID, dtype: int64